In [1]:
import torch
torch.cuda.is_available()

# import os
# os.environ['CUDA_VISIBLE_DEVICES']='1'
# # os.environ['CUDA_VISIBLE_DEVICES']='0,1'
# os.environ['CUDA_LAUNCH_BLOCKING']='1'

# torch.cuda.set_device(0)
# print(torch.cuda.current_device())
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
# import torch
import argparse
import numpy as np

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', './config/cv11Lu_asr_lstm4atthead_allvocab-whisperclass.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'LAS_ckpt/')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', None)
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',0)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', False)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 9)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', False)
    setattr(paras, 'binaryClass', True)
#     force_cudnn_initialization()
    print(torch.multiprocessing.get_start_method())
#     torch.multiprocessing.set_start_method('spawn',force=True)
    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            from test_asr import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        elif paras.binaryClass:
            # from train_whisperclass import Solver
            from Whisper_Biclass_concat_train import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))

    for idx in range(0,5):
        paras.config = f'./config/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-{idx+1}.yaml'#latest7k_4lstm.pth
        # setattr(paras, 'load', f'./ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-{idx+1}_sd0/whisper8k.pth')
        #要用vag01要改load_ckpt
        config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)
        solver = Solver(config, paras, mode)


        solver.load_data()
        # solver.load_data_2()
        # solver.print_model()
        solver.set_model()
        solver.exec()
        del solver

In [2]:
main()

spawn

Using train mode

[INFO] Exp. name : cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1_sd0                                   
[INFO] Loading data... large dataset may took a while.                                                     


c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_r

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-1-15']
Mozillacv11Dataset CTT5-1-15 found wav data: 51
text len: 51
remove None, then wav data: 51, text len: 51
[VAGDataset] path: data_process, split: ['CTT5-5-15', 'CTT5-2-15', 'CTT5-3-15', 'CTT5-4-15']
Mozillacv11Dataset CTT5-5-15 found wav data: 45
Mozillacv11Dataset CTT5-2-15 found wav data: 41
Mozillacv11Dataset CTT5-3-15 found wav data: 50
Mozillacv11Dataset CTT5-4-15 found wav data: 45
text len: 181
remove None, then wav data: 181, text len: 181
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=False is unstable?
# Optimizer
[INFO] Optim.spec.| Algo. = Adadelta	| Lr = 1.0	 (Scheduler = fixed)| Scheduled sampling = False           
# Enable AMP if needed
fc.weight
fc.bias
Training End-to-end ASR system
[INFO] Total training steps 1.0

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a Be

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-2-15']
Mozillacv11Dataset CTT5-2-15 found wav data: 41
text len: 41
remove None, then wav data: 41, text len: 41
[VAGDataset] path: data_process, split: ['CTT5-1-15', 'CTT5-5-15', 'CTT5-3-15', 'CTT5-4-15']
Mozillacv11Dataset CTT5-1-15 found wav data: 51
Mozillacv11Dataset CTT5-5-15 found wav data: 45
Mozillacv11Dataset CTT5-3-15 found wav data: 50
Mozillacv11Dataset CTT5-4-15 found wav data: 45
text len: 191
remove None, then wav data: 191, text len: 191
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=False is unstable?
# Optimizer
[INFO] Optim.spec.| Algo. = Adadelta	| Lr = 1.0	 (Scheduler = fixed)| Scheduled sampling = False           
# Enable AMP if needed
fc.weight
fc.bias
Training End-to-end ASR system
[INFO] Total training steps 1.0

...

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-3-15']
Mozillacv11Dataset CTT5-3-15 found wav data: 50
text len: 50
remove None, then wav data: 50, text len: 50
[VAGDataset] path: data_process, split: ['CTT5-1-15', 'CTT5-2-15', 'CTT5-5-15', 'CTT5-4-15']
Mozillacv11Dataset CTT5-1-15 found wav data: 51
Mozillacv11Dataset CTT5-2-15 found wav data: 41
Mozillacv11Dataset CTT5-5-15 found wav data: 45
Mozillacv11Dataset CTT5-4-15 found wav data: 45
text len: 182
remove None, then wav data: 182, text len: 182
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=False is unstable?
# Optimizer
[INFO] Optim.spec.| Algo. = Adadelta	| Lr = 1.0	 (Scheduler = fixed)| Scheduled sampling = False           
# Enable AMP if needed
fc.weight
fc.bias
Training End-to-end ASR system
[INFO] Total training steps 1.0

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a Be

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-4-15']
Mozillacv11Dataset CTT5-4-15 found wav data: 45
text len: 45
remove None, then wav data: 45, text len: 45
[VAGDataset] path: data_process, split: ['CTT5-1-15', 'CTT5-2-15', 'CTT5-3-15', 'CTT5-5-15']
Mozillacv11Dataset CTT5-1-15 found wav data: 51
Mozillacv11Dataset CTT5-2-15 found wav data: 41
Mozillacv11Dataset CTT5-3-15 found wav data: 50
Mozillacv11Dataset CTT5-5-15 found wav data: 45
text len: 187
remove None, then wav data: 187, text len: 187
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=False is unstable?
# Optimizer
[INFO] Optim.spec.| Algo. = Adadelta	| Lr = 1.0	 (Scheduler = fixed)| Scheduled sampling = False           
# Enable AMP if needed
fc.weight
fc.bias
Training End-to-end ASR system
[INFO] Total training steps 1.0

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a Be

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-5-15']
Mozillacv11Dataset CTT5-5-15 found wav data: 45
text len: 45
remove None, then wav data: 45, text len: 45
[VAGDataset] path: data_process, split: ['CTT5-1-15', 'CTT5-2-15', 'CTT5-3-15', 'CTT5-4-15']
Mozillacv11Dataset CTT5-1-15 found wav data: 51
Mozillacv11Dataset CTT5-2-15 found wav data: 41
Mozillacv11Dataset CTT5-3-15 found wav data: 50
Mozillacv11Dataset CTT5-4-15 found wav data: 45
text len: 187
remove None, then wav data: 187, text len: 187
Setup ASR model and optimizer 
# Losses
# Note: zero_infinity=False is unstable?
# Optimizer
[INFO] Optim.spec.| Algo. = Adadelta	| Lr = 1.0	 (Scheduler = fixed)| Scheduled sampling = False           
# Enable AMP if needed
fc.weight
fc.bias
Training End-to-end ASR system
[INFO] Total training steps 1.0